# memo

https://www.evernote.com/view/notebook/27d0d4e1-623f-46ef-80fc-600b075aa846?locale=ja#st=p&x=future%2520contango&n=27d0d4e1-623f-46ef-80fc-600b075aa846

[A Very Different Kind of Trend Model | Following the Trend](http://www.followingthetrend.com/2016/11/a-very-different-kind-of-trend-model/ "A Very Different Kind of Trend Model | Following the Trend")

[原油ETFの影響でWTI原油はコンタンゴになりやすい？！ - 原油投資情報サイト](http://crudeoil123.com/contango_due_to_crude_oil_etf/ "原油ETFの影響でWTI原油はコンタンゴになりやすい？！ - 原油投資情報サイト")


In [ ]:
from quantopian.research.experimental import continuous_future, history
import statsmodels.api as sm 
import statsmodels.tsa as tsa 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import matplotlib.cm as cm


root_symbol = 'NG'
future0 = continuous_future(root_symbol)
future1 = continuous_future(root_symbol, offset=1, )

futures = history([future0,future1], fields=['price', 'contract'],frequency="daily",
                  start="2015-1-1", end="2018-2-1")

futures.minor_axis = futures.minor_axis.map(lambda x: x.root_symbol + str(x.offset))
f0 = futures.minor_axis[0]
f1 = futures.minor_axis[1]
pricing = futures["price"]

pricing["steepness"] = pricing[f1] / pricing[f0] 

def get_expiration_date(contract):
    return contract.expiration_date

contract = futures['contract']

contract["expiration_date"] = contract[f0].apply(get_expiration_date)
contract["date"] = contract.index # memo1
contract["remaining days"] = (contract["expiration_date"] - contract["date"]).dt.days

pricing["remaining days"] = contract["remaining days"]

pricing["implied steepness"] = pricing["steepness"]/pricing["remaining days"]

In [ ]:
contract["expiration_date1"] = contract[f1].apply(get_expiration_date)
contract["remaining days1"] = (contract["expiration_date1"] - contract["date"]).dt.days
contract["contract term"] = contract["remaining days1"] - contract["remaining days"]


In [ ]:
pricing["contract term"]  = contract["contract term"] 

In [ ]:
pricing["annualize steepness"] = pricing["steepness"]*pricing["remaining days"]/pricing["contract term"]

In [ ]:
pricing[["steepness", "annualize steepness"]].plot(secondary_y ="steepness")

In [ ]:
plt.plot(pricing["NG0"] * pricing["remaining days"]/pricing["contract term"] + pricing["NG1"] * (1-pricing["remaining days"]/pricing["contract term"] ))
plt.plot(pricing["NG0"], c='red')


In [ ]:
pricing["synsetic NG0"] = pricing["NG0"] * pricing["remaining days"]/pricing["contract term"] + pricing["NG1"] * (1-pricing["remaining days"]/pricing["contract term"] )

In [ ]:
pricing[["synsetic NG0", "NG0", "NG1", "remaining days"]].tail(300).plot(secondary_y="remaining days")

In [ ]:
pricing["diff steepness"] = pricing["steepness"] / pricing["steepness"].shift(1)

In [ ]:
pricing[["synsetic NG0", "diff steepness"]].plot(secondary_y = "diff steepness")

In [ ]:
start = pricing.index[0]
end = pricing.index[-1]
ung = get_pricing('UNG', start_date=start, end_date=end, fields='price')

In [ ]:
plt.plot(ung.tail(100))
plt.plot(pricing['synsetic NG0'].tail(100))

In [ ]:
dftest = pricing.copy()
dftest = dftest[['NG0', 'NG1', 'remaining days']]
dftest['NG0'] = dftest['NG0'].apply(np.log)
dftest['NG1'] = dftest['NG1'].apply(np.log)
dftest['diff']= (dftest['NG1']-dftest['NG0'])

In [ ]:
dftest['timedecays'] = dftest['diff']/dftest['remaining days']

In [ ]:
dftest[['diff', 'timedecays']].plot()

In [ ]:
dftest.plot(x="diff", y="timedecays", kind="scatter", c=range(len(dftest.index)))#,  ylim=([-0.03,0.035])

In [ ]:
dftest[["NG0","NG1", "timedecays"]].plot(secondary_y="timedecays")

In [ ]:
dftest["timedecays"].hist(bins=100)

In [ ]:
dftest[dftest['timedecays'] < 0]["remaining days"].hist()

In [ ]:
dftest[(dftest['timedecays'] > 0.00) & (dftest['timedecays'] < 0.01) & (dftest["remaining days"] == 5)]

In [ ]:
dftest.loc["2017-12-22":].head(30)[['NG0', 'NG1', 'diff']].plot(secondary_y = 'diff')

In [ ]:
test = pricing[['NG0', 'NG1', 'remaining days']]


In [ ]:
test['NG0 change'] = test["NG0"].pct_change()
test['NG1 change'] = test["NG1"].pct_change()


In [ ]:

idx = pd.IndexSlice

test.groupby(by="remaining days")[["NG0 change", "NG1 change"]].describe().loc[idx[:,['count']],:]



In [ ]:
contract.groupby(by="NG0")["expiration_date"].count()

In [ ]:
contract

In [ ]:
dir(future0)

In [ ]:
future0.sid

In [ ]:
contract['NG0'][0].sid


In [ ]:
contract = futures['contract'].copy()

In [ ]:
s = contract.groupby(by="NG0")[["NG0"]].count()

In [ ]:
contract.merge(s, how='outer', left_on="NG0", right_index=True)

In [ ]:
contract["roll date?"] = contract["NG0"] != contract["NG0"].shift(-1)

In [ ]:
contract

In [ ]:
def nan_or_1(x):
    if x:
        return np.nan
    else: 
        return 1


v = contract["roll date?"].apply(nan_or_1).iloc[::-1] ## iloc[::-1] は index を reverse する一番簡単な方法
n = np.isnan(v)
a = ~n
c = np.cumsum(a)
d = np.diff(np.concatenate(([0.], c[n])))
v[n] = -d
np.cumsum(v).iloc[::-1]

contract["roll date?"] = np.cumsum(v).iloc[::-1]


In [ ]:
contract